In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import tensorflow as tf
import numpy as np
import math
import random
import pandas as pd
import csv
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
def conv3d(x, W):
    return tf.nn.conv3d(x, W, strides=[1,1,1,1,1], padding='SAME')

In [3]:
def maxpool3d(x):
    #                        size of window         movement of window
    return tf.nn.max_pool3d(x, ksize=[1,2,2,2,1], strides=[1,2,2,2,1], padding='SAME')

In [4]:
def convolutional_neural_network(x):
    initializer = tf.contrib.layers.xavier_initializer()
    weights = {'W_conv1':tf.Variable(initializer([3,3,3,1,8]), name="W_conv1"),
               'W_conv2':tf.Variable(initializer([3,3,3,8,16]), name="W_conv2"),
               'W_conv3':tf.Variable(initializer([3,3,3,16,32]), name="W_conv3"),
               'W_conv4':tf.Variable(initializer([3,3,3,32,64]), name="W_conv4"),
               'W_conv5':tf.Variable(initializer([3,3,3,64,128]), name="W_conv5"),
               'W_fc':tf.Variable(initializer([2304,1024]), name="W_fc"),
               'W_out':tf.Variable(initializer([1024, n_classes]), name="W_out")}

    biases = {'b_conv1':tf.Variable(initializer([8]), name="b_conv1"),
               'b_conv2':tf.Variable(initializer([16]), name="b_conv2"),
              'b_conv3':tf.Variable(initializer([32]), name="b_conv3"),
              'b_conv4':tf.Variable(initializer([64]), name="b_conv4"),
              'b_conv5':tf.Variable(initializer([128]), name="b_conv5"),
               'b_fc':tf.Variable(initializer([1024]), name="b_fc"),
               'b_out':tf.Variable(initializer([n_classes]), name="b_out")}

    x = tf.reshape(x, shape=[-1, IMG_SIZE_PX, IMG_SIZE_PX, SLICE_COUNT, 1])

    conv1 = tf.nn.relu(conv3d(x, weights['W_conv1']) + biases['b_conv1'])
    conv1 = tf.nn.dropout(conv1, dropout_rate)
    conv1 = maxpool3d(conv1)
    
    conv2 = tf.nn.relu(conv3d(conv1, weights['W_conv2']) + biases['b_conv2'])
    conv2 = tf.nn.dropout(conv2, dropout_rate)
    conv2 = maxpool3d(conv2)

    conv3 = tf.nn.relu(conv3d(conv2, weights['W_conv3']) + biases['b_conv3'])
    conv3 = tf.nn.dropout(conv3, dropout_rate)
    conv3 = maxpool3d(conv3)

    conv4 = tf.nn.relu(conv3d(conv3, weights['W_conv4']) + biases['b_conv4'])
    conv4 = tf.nn.dropout(conv4, dropout_rate)
    conv4 = maxpool3d(conv4)

    conv5 = tf.nn.relu(conv3d(conv4, weights['W_conv5']) + biases['b_conv5'])
    conv5 = tf.nn.dropout(conv5, dropout_rate)
    conv5 = maxpool3d(conv5)
    

    fc = tf.reshape(conv5,[-1, 2304])
    #fc = tf.nn.relu(tf.matmul(fc, weights['W_fc'])+biases['b_fc'])
    fc = tf.matmul(fc, weights['W_fc'])+biases['b_fc']
    fc = tf.nn.dropout(fc, dropout_rate)

    output = tf.add(tf.matmul(fc, weights['W_out']),biases['b_out'],name="output")

    return output[0]

In [5]:
def train_neural_network(x):

    
    prediction = convolutional_neural_network(x)
    cost = tf.reduce_mean(tf.square(prediction-y),name="cost")
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    #accuracy = tf.contrib.metrics.streaming_pearson_correlation(prediction, tf.cast(y,tf.float32))[0]
    saver = tf.train.Saver(save_relative_paths=True)
    
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.local_variables_initializer())

        for epoch in range(hm_epochs):
            avg_cost = 0.
            
            index = [ind for ind in range(1,train_batch+1)]
            original = index[:]
            random.shuffle(index)
            
            for original_index in range (1, train_batch+1):
                current_batch = index[original_index-1]
                batch_data = np.load(folder + 'batch({},{})-{}-{}-{}.npy'.format(current_batch*batch_size+1,
                                    current_batch*batch_size+batch_size,IMG_SIZE_PX,IMG_SIZE_PX,SLICE_COUNT))

                for data in batch_data:
                    X = data[0]
                    Y = data[1]
                    _, c, p = sess.run([optimizer, cost, prediction], feed_dict={x: X, y: Y})
                    avg_cost += c / (train_batch * batch_size)

            print('Epoch', epoch+1, 'completed out of',hm_epochs,'cost:',avg_cost)
            
            avg_valicost = 0.
            pred_matrix = np.array([]).reshape(0,1)
            label_matrix = np.array([]).reshape(0,1)
            for current_validation in range (train_batch, train_batch+validation_batch):
                validation_data = np.load(folder + 'batch({},{})-{}-{}-{}.npy'.format(current_validation*batch_size+1,
                                            current_validation*batch_size+batch_size,IMG_SIZE_PX,IMG_SIZE_PX,SLICE_COUNT))
                for data in validation_data:
                    c, predict_value = sess.run([cost, prediction], feed_dict={x:data[0], y:data[1]})
                    predict_value = predict_value.reshape(1,1)
                    avg_valicost += c / (validation_batch * batch_size)
                    label_value = data[1].reshape(1,1)
                    pred_matrix=np.concatenate((pred_matrix, predict_value), axis=0)
                    label_matrix=np.concatenate((label_matrix, label_value), axis=0)
                    
            print("avgvalicost", avg_valicost)
            print(np.concatenate((pred_matrix.T, label_matrix.T),axis=0).shape)
            evaluated_accuracy = np.corrcoef(np.concatenate((pred_matrix.T, label_matrix.T), axis=0))
            print('pearson:',evaluated_accuracy)

            if ((epoch+1)%epoch_save==0):
                filename = '/home/lvruyi/regression_small_epoch_' + str(epoch+1)
                saver.save(sess, filename)
                test_data = np.load(folder + 'batch(1473,1482)-65-65-55.npy')
                #test_data = np.load(folder + 'batch(2065,2072)-65-65-55.npy')
                for tdata in test_data:
                    label_value = tdata[1]
                    predicted_value = sess.run(prediction, feed_dict={x: tdata[0], y:tdata[1]})
                    print ("label value:", label_value, ";    estimated value:", predicted_value)

In [ ]:
IMG_SIZE_PX = 65
SLICE_COUNT = 55
n_classes = 1
#train_batch = 15
train_batch = 75
batch_size = 16
#validation_batch = 4
validation_batch = 17
labels_file = pd.read_csv('FYP_Phenotypic2noxrotate.csv')

tf.reset_default_graph()
x = tf.placeholder('float',name="x")
y = tf.placeholder('float',name="y")

learning_rate = 0.00007
dropout_rate = 0.8
hm_epochs = 20000
epoch_save= 5
#"combined2\\"
folder = '/home/lvruyi/combined2noxrotate/'

#calculate_mean()`

#shuffle()

"""
for batch in range (0, train_batch+validation_batch-1):
    combine_preprocess(batch*batch_size+1, batch*batch_size+batch_size)

combine_preprocess(2065, 2072)
"""
                                     
train_neural_network(x)
